# Download Waveform Accelerometer
# Ambient Noise (Midnight)
Working Group Member 3 BMKG

***Authors:***
1.  Iman Fatchurochman, S.Si, M.DM
2.  Nova Heryandoko, S.Si, M.Si
3.  Ajat Sudrajat, M.Sc
4.  Bayu Pranata, M.Si
5.  Arif Nurokhim
6.  Kian Purna Sinki, SST, M.DM
7.  Melky Adi Kurniawan, S.Si
8.  Robby Wallansha, S.Tr
9.  Abdul Rosid, S.Tr, M.DM
10. Muhammad Fahmi Nugraha, S.Tr
11. Sesar Prabu Dwi Sriyanto
12. Ade Andika Saputra, S.Tr.Geof.
13. Rosi Budi Kurniawan, S.Tr.Geof.
14. Angga Wijaya, S.Tr.Geof.

In [ ]:
# Import Module
import os
import sys
from obspy import UTCDateTime
from obspy.clients.fdsn import Client

In [ ]:
# Get parameter
dirf = sys.argv[1] # 20190106T172721_12675_0234_56_66

# Server FDSN Accelerometer
client = Client("http://172.19.3.212:8080/")

# Datetime noise 17-19 UTC
ot_noise = dirf.split("T")
ot_noise_start = UTCDateTime(f"{ot_noise[0]}T17:00:00")
ot_noise_end = UTCDateTime(f"{ot_noise[0]}T19:00:00")

# Directory
if not os.path.isdir(dirf):
    os.mkdir(dirf)
if not os.path.isdir(dirf + '/noises'):
    os.mkdir(dirf + '/noises')
if not os.path.isdir(dirf + '/earthquakes'):
    os.mkdir(dirf + '/earthquakes')

# Read data station txt
file = open(dirf + '/earthquakes/station_fix.txt')
sta = file.readlines()
file.close()

for i in range(len(sta)):
    sta[i] = sta[i].split()

# Create use epicenter txt
staUpdate = open(dirf + '/sta_noise.txt', 'w')

for s in sta:
    if os.path.exists(dirf + '/noises/' + s[0] + 'noise.mseed'):
        continue
    else:
        try:
            st_noise = client.get_waveforms("IA", s[0], "*", "HN?", ot_noise_start, ot_noise_end)
        except:
            try:
                st_noise = client.get_waveforms("IA", s[0], "*", "SL?", ot_noise_start, ot_noise_end)
            except:
                print(s[0], ot_noise_start, ot_noise_end, 'No Data')
                continue

    st_noise.merge(method=1, fill_value="interpolate")

    if st_noise.count() < 3:
        print(s[0], 'Not full')
        continue

    st_noise.write(dirf + '/noises/' + s[0] + 'noise.mseed')
    st_noise.plot(equal_scale='False', outfile=dirf + "/noises/waveform_" + s[0] + "noise.png")

    staUpdate.write(f"{s[0]}\n")

staUpdate.close()